In [20]:
class Empty(Exception):
    """Error attempting to access an element from an empty container"""
    pass

class PriorityQueueBase:
    """Abstract base class for a priority queue."""

    class _Item:
        """Lightweight composite to store prirority queue items."""
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k
            self._value = v 

        def __lt__(self, other):
            return self._key < other._key
    
    def is_empty(self):
        """Return True if the priority queue is empty."""
        return len(self) == 0

class _DoublyLinkedBase:
    '''A base class providing a doubly linked list representation '''
    
    class _Node:
        ''' Lightweight, nonpublic class for storing a doubly linked node. '''
        __slots__ = '_element', '_prev', '_next'  # streamline memory

        def __init__(self, element, prev, next):
            self._element = element
            self._prev = prev
            self._next = next
    
    def __init__(self):
        '''Create an empty list'''
        self._header = self._Node(None, None, None)
        self._trailer = self._Node(None, None, None)
        self._header._next = self._trailer                  # trailer is after header
        self._trailer._prev = self._header
        self._size = 0

    
    def __len__(self):
        '''Return the number of elements in the list.'''
        return self._size
    
    def is_empty(self):
        '''Return True if list is empty'''
        return self._size == 0

    def _insert_between(self, e, pred, succ):
        '''Add element e between two existing nodes and return new node. '''
        newest = self._Node(e, pred, succ) # linked to neighbors
        pred._next = newest
        succ._prev = newest
        self._size += 1
        return newest
    
    def _delete_node(self, node):
        '''Delete nonsentinel node from the list and return its element.'''
        pred = node._prev
        succ = node._next
        pred._next = succ
        succ._prev = pred
        self._size -= 1
        element = node._element                            # record delete element
        node._prev = node._next = node._element = None     # deprecate node
        return element

class PositionalList(_DoublyLinkedBase):
    '''A sequential container of elements allowing positional access.'''

    # ---------------------------- nested Position class -----------------------------
    class Position : 
        ''' An abstraction representing the location of a single element.'''
        def __init__(self, container, node):
            '''Constructor should not be invoked by user.'''
            self._container = container
            self._node = node
        
        def element(self):
            '''Return the element stored at this Position.'''
            return self._node._element
        
        def __eq__(self, other):
            '''Return True if other is a Position representing the same location'''
            return type(other) is type(self) and other._node is self._node
        
        def __ne__(self, other):
            '''Return True if other does not represent the same location.'''
            return not (self == other)                # opposite of __eq__
        
    # ---------------------------- utility method -------------------------------------
    def _validate(self, p):
        '''Return position"s node, or raise appropriate error if position is invalid.'''
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._next is None:
            raise ValueError('p is no longer valid')
        return p._node
    
    def _make_position(self, node):
        '''Return Position instance for given node (or None if sentinel).'''
        if node is self._header or node is self._trailer:
            return None
        else :
            return self.Position(self, node)
    
    # ---------------------------- accessors ------------------------------------------

    def first(self):
        '''Return the first Position in the list (or None if list is empty).'''
        return self._make_position(self._header._next)
    
    def last(self):
        '''Return the last Position in the list (or None if list is empty).'''
        return self._make_position(self._trailer._prev)
    
    def before(self, p):
        '''Return the Position just before Position p (or None if p is first).'''
        node = self._validate(p)
        return self._make_position(node._prev)
    
    def after(self, p):
        '''Return the Position just after Position p (or None if p is last).'''
        node = self._validate(p)
        return self._make_position(node._next)
    
    def __iter__(self):
        '''Generate a forward iteration of the elements of the list.'''
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)

    # ---------------------------- mutators ------------------------------------------
    def _insert_between(self, e, pred, succ):
        '''Add element between existing nodes and return new Position.'''
        node = super()._insert_between(e, pred, succ)
        return self._make_position(node)
    
    def add_first(self, e):
        '''Insert element e at the front of the list and return new Position.'''
        return self._insert_between(e, self._header, self._header._next)
    
    def add_last(self, e):
        '''Insert element e at the back of the list and return new Position.'''
        return self._insert_between(e, self._trailer._prev, self._trailer)
    
    def add_before(self, p, e):
        '''Insert element e into list before Position p and return new Position.'''
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)

    def add_after(self, p, e):
        '''Insert element e into list after Position p and return new Position.'''
        original = self._validate(p)
        return self._insert_between(e, original, original._next)
    
    def delete(self, p):
        '''Remove and return the element at Position p.'''
        original = self._validate(p)
        return self._delete_node(original)
    
    def replace(self, p, e):
        '''
            Replace the element at Position p with e.
            Return the element formerly at Position p.
        '''
        original = self._validate(p)
        old_value = original._element
        original._element = e
        return old_value

class UnsortedPriorityQueue(PriorityQueueBase): 
    """A min-oriented priority queue implemented with an unsorted list."""
    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = PositionalList()

    def __len__(self):
        return len(self._data)
    
    def _find_min(self):                                    # nonpublic utility
        """Return Position of item with minimum key."""
        if self.is_empty():                                 # is_empty inherited from base class
            raise Empty('Priority queue is empty')
        small = self._data.first()
        walk = self._data.after(small)
        while walk is not None:
            if walk.element() < small.element():
                small = walk
            walk = self._data.after(walk)
        return small

    def add(self, key, value):
        """Add a key-value pair."""
        self._data.add_last(self._Item(key, value))

    def md_add(self, key, value):
        """modiffy a key-value add"""
        if self.is_empty():
            self._data.add_last(self._Item(key, value))
        else:
            walk = self._data.first()
            while walk is not None:
                if walk.element() < self._Item(key, value):
                    walk = self._data.after(walk)
                self._data._insert_between(self._Item(key, value), walk, self._data.after(walk))
        return self._Item(key, value)

    def md_find_min(self):
        return self._data.first()

    def min(self):
        """Return but do not remove (k, v) tuple with minimum key."""
        p = self._find_min()
        item = p.element()
        return (item._key, item._value)

    def remove_min(self):
        """Remove and return (k, v) tuple with minmum key."""
        p = self._find_min()
        item = self._data.delete(p)
        return (item._key, item._value)

# Practice 
us_pq = UnsortedPriorityQueue()
us_pq.add(3, 'A')
us_pq.add(2, 'B')
us_pq.add(1, 'C')
print(us_pq.min())
print(us_pq.remove_min())
print(us_pq.min())

# Practice R-9.5 The min method for the UnsortedPriorityQueue class executes in O(n)
# time , as analyzed in Table 9.2. Give a simple modification to the class so that min 
# runs in O(1) time.
# us_pq = UnsortedPriorityQueue()
# us_pq.md_add(3, 'A')
# us_pq.md_add(2, 'B')
# us_pq.md_add(1, 'C')
# print(us_pq.md_find_min())

# -------- SortedPriorityQueue -----------------------
class SortedPriorityQueue(PriorityQueueBase):  # base class defined _Item 
    """A min-oriented priority queue implementd with a sorted list."""

    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = PositionalList()

    def __len__(self):
        """Return the number of items in the priority queue."""
        return len(self._data)
    
    def add(self, key, value): # O(n)
        """Add a key-value pair"""
        newest = self._Item(key, value)          # make new item instance
        walk = self._data.last()
        while walk is not None and newest < walk.element():
            walk = self._data.before(walk)
        if walk is None:
            self._data.add_first(newest)
        else:
            self._data.add_after(walk, newest)
    
    def min(self):
        """Return but do not remove (k, v) tuple with minimum key."""
        if self.is_empty():
            raise Empty('Priority queue is empty')
        p = self._data.first() 
        item = p.element()
        return (item._key, item._value)


    def remove_min(self):
        """Remove and return (k, v) tuple with minimum key."""
        if self.is_empty():
            raise Empty('Priority queue is empty.')
        item = self._data.delete(self._data.first())
        return (item._key, item._value)

# Practice 
# st_pq = SortedPriorityQueue()
# st_pq.add(5, 'A')
# st_pq.add(4, 'B')
# st_pq.add(2, 'C')
# st_pq.min()
# st_pq.remove_min()
# st_pq.min()


# -------------------- HeapPriorityQueue -------------------------- 
class HeapPriorityQueue(PriorityQueueBase):      # base class defines _Item 
    """A min-oriented priority queue implemented with a binary heap."""
    # ---------------- nonpublic behaviors ------------------
    def _parent(self, j):
        return (j-1) // 2
    
    def _left(self, j):
        return 2*j + 1

    def _right(self, j):
        return 2*j + 2 

    def _has_left(self, j):
        return self._left(j) < len(self._data)       # index beyond end of list ?

    def _has_right(self, j):
        return self._right(j) < len(self._data)      # index beyond end of list ?

    def _swap(self, i, j):
        """Swap the elements at indices i and j of array."""
        self._data[i], self._data[j] = self._data[j], self._data[i]

    def _upheap(self, j):
        parent = self._parent(j) 
        if j >= 0 and self._data[j] < self._data[parent]:
            self._swap(j, parent)

    def _downheap(self, j):
        if self._has_left(j):
            left = self._left(j)
            small_child = left 
            if self._has_right(j):
                right = self._right(j)
                if self._data[right] < self._data[left]:
                    small_child = right 
            if self._data[small_child] < self._data[j]:
                self._swap(j, small_child)
                self._downheap(small_child)
    
    # ------------------- public behaviors -------------------
    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = [] 
    
    def __len__(self):
        """Return the number of items inthe priority queue."""
        return len(self._data)
    
    def add(self, key, value):
        """Add a key-value pair to the priority queue."""
        self._data.append(self._Item(key, value))
        self._upheap(len(self._data)-1)
    
    def min(self):
        """Return but do not remove (k, v) tuple with minimum key.
        Raise Empty exception if empty.
        """ 
        if self.is_empty():
            raise Empty('Priority queue is empty')
        item = self._data[0]
        return (item._key, item._value)

    def remove_min(self):
        """Remove and return (k,v) tuple with minimum key.
        Raise Empty exception if empty."""
        if self.is_empty():
            raise Empty('Priority queue is empty')
        print('self._data[0] is ->', self._data[0])
        self._swap(0, len(self._data)-1)            # put minimum item at the end
        item = self._data.pop()                     # and remove it from the list;
        self._downheap(0)                           # then fix new root
        return (item._key, item._value)

    def traverse(self, j, result):
        left = right = None
        if len(self._data) != 0:
            if j == 0:
                result.append(self._data[0]._key)
            if self._has_left(j):
                left = self._left(j)
                result.append(self._data[left]._key)
            if self._has_right(j):
                right = self._right(j)
                result.append(self._data[right]._key)
            if left:
                self.traverse(left, result)
            if right:
                self.traverse(right, result)
        print('result is ->', result)



# heap_pri = HeapPriorityQueue()
# heap_pri.add(1, "A")
# heap_pri.add(4, "B")
# heap_pri.add(3, "Y")
# heap_pri.add(2, "G")
# heap_pri.add(8, "Q")
# heap_pri.add(5, "I")
# heap_pri.traverse(0, [])


# --------------------------------------------------------
# Illustrate the execution of selection-sort algorithm on the  following input sequence
def selection_sort(input_seq):
    heap_que = HeapPriorityQueue()
    for i in input_seq:
        heap_que.add(str(i),i)
    heap_que.traverse(0, [])
    result = []
    while len(heap_que) != 0:
        result.append(heap_que.remove_min())
    print('result is ->', result)

input_seq = [2, 1 ,3, 6, 7, 4]
selection_sort(input_seq)


class _DoublyLinkedBase:
    """A base class providing a doubly linked list representations."""

    class _Node:
        __slots__ = '_element', '_prev', '_next'
        
        def __init__(self, element, prev, next):
            self._element = element 
            self._prev = prev 
            self._next = next 
    
    def __init__(self):
        """Create an empty list."""
        self._header = self._Node(None, None, None)       # sentinel node
        self._trailer = self._Node(None, None, None)      # sentinel node
        self._header._next = self._trailer
        self._trailer._prev = self._header 
        self._size = 0
    
    def __len__(self):
        return self._size 

    def is_empty(self):
        """Return True if list is empty"""
        return self._size == 0
    
    def _insert_between(self, e, predecessor, successor):
        """Add element e between two existing nodes and return new node."""
        newest = self._Node(e, predecessor, successor)
        predecessor._next = newest
        successor._prev = newest
        self._size += 1
        return newest
    
    def _delete_node(self, node):
        predecessor = node._prev
        successor = node._next
        predecessor._next = successor
        successor._prev = predecessor
        element = node._element
        node._prev = node._next = node._element = None 
        self._size -= 1
        return element


class PositionalList(_DoublyLinkedBase):
    """A sequential container of elements allowing positional access."""

    # ------------------- nested Position class -------------------------
    class Position:
        """An abstraction representing the location of a single element."""
        def __init__(self, container, node):
            """Constructor should not be invoked by user."""
            self._container = container 
            self._node = node 
        
        def element(self):
            """Return the element stored at this Position."""
            return self._node._element
    
        def __eq__(self, other):
            """Return True if other is a Position representing the same location."""
            return type(other) is type(self) and other._node is self._node 
    
        def __ne__(self, other):
            """Return True if other does not represent the same location."""
            return not(self == other) 
    
    # ------------------- utility method ------------------------------------
    def _validate(self, p):
        """Return position's node, or raise appropriate error if invalid."""
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._next is None:                             # convention for deprecated nodes
            raise ValueError('p is no longer valid')
        return p._node 
    
    def _make_position(self, node):
        """Return Position instance for given node (or None if sentinel)."""
        if node is self._header or node is self._trailer:
            return None                                       # boundary violation
        else:
            return self.Position(self, node)                  # legitimate position
    
    # ----------------------- accessors -------------------------------------
    def first(self):
        """Return the first Position in the list (or None if list is empty)."""
        return self._make_position(self._header._next)
    
    def last(self):
        """Return the last Position in the list (or None if list is empty)."""
        return self._make_position(self._trailer._prev)
    
    def before(self, p):
        """Return the Position just before Position p (or None if p is first)."""
        node = self._validate(p)
        return self._make_position(node._prev)
    
    def after(self, p):
        """Return the Position just after Position p (or None if p is last)."""
        node = self._validate(p)
        return self._make_position(node._next)

    def __iter__(self):
        """Generate a forward iteration of the elements of the list."""
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)
    
    # ---------------------- mutators --------------------------------
    # override inherited version to return Position, rather than Node
    def _insert_between(self, e, predecessor, successor):
        """Add element between existing nodes and return new Position."""
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)
    
    def add_first(self, e):
        """Insert element e at the front of the list and return new Position."""
        return self._insert_between(e, self._header, self._header._next)
    
    def add_last(self, e):
        """Insert element at the back of the list and return new Position."""
        return self._insert_between(e, self._trailer._prev, self._trailer)
    
    def add_before(self, p, e):
        """Insert element e into list before Position p and return new Position."""
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)
    
    def add_after(self, p, e):
        """Insert element e into list after Position p and return new Position."""
        original = self._validate(p)
        return self._insert_between(e, original, original._next)
    
    def delete(self, p):
        """Remove and return the element at Position p."""
        original = self._validate(p)
        return self._delete_node(original)
    
    def replace(self, p, e):
        """Replace the element at Position p with e.
        Return the element formerly at Position p
        """
        original = self._validate(p)
        old_value = original._element
        original._element = e
        return old_value

    def __max__(self):
        if self.is_empty():
            return None
        p = self.first()
        max_element = p.element()

        while self.after(p):
            p = self.after(p)
            if p.element() > max_element:
                max_element = p.element()
        return max_element

    def find(self, e):
        p = self.first()
        while self.after(p) and p.element() != e:
            p = self.after(p)
        if p.element() != e:
            return None 
        return p

    def find_recursive(self, e, p = None): # Space usage is 
        if not p:
            p = self.first()                                # space: O(1)
        if not self.after(p):
            if p.element() == e:
                return p
            return None
        if p.element() != e and self.after(p):
            return self.find_recursive(e, self.after(p))    # space: O(size - 1), size of the PositionalList
        


# C is a positional list 
# P is a Priority Queue 

# Selection Sort   
# def pq_sort(C): # assume C is a positional list
#     """Sort a collection of elements stored in a positional list."""
#     n = len(C)
#     P = PriorityQueueBase()
#     for j in range(n):
#         element = C.delete(C.first()) # O(1)
#         P.add(element, element)
#     for j in range(n):
#         (k, v) = P.remove_min() # O(len(P))
#         C.add_last(v)



(1, 'C')
(1, 'C')
(2, 'B')
result is -> ['1', '2', '3', '6', '7']
result is -> ['1', '2', '3', '6', '7']
result is -> ['1', '2', '3', '6', '7']
result is -> ['1', '2', '3', '6', '7', '4']
result is -> ['1', '2', '3', '6', '7', '4']
result is -> ['1', '2', '3', '6', '7', '4']
self._data[0] is -> <__main__.PriorityQueueBase._Item object at 0x1081d7d60>
self._data[0] is -> <__main__.PriorityQueueBase._Item object at 0x1081d7cd0>
self._data[0] is -> <__main__.PriorityQueueBase._Item object at 0x10838b8e0>
self._data[0] is -> <__main__.PriorityQueueBase._Item object at 0x1077f5d00>
self._data[0] is -> <__main__.PriorityQueueBase._Item object at 0x1077f5be0>
self._data[0] is -> <__main__.PriorityQueueBase._Item object at 0x1077f5700>
result is -> [('1', 1), ('2', 2), ('3', 3), ('4', 4), ('6', 6), ('7', 7)]
False


In [2]:
print(3 // 1)

3
